# InfernoSaber 

Hi, here you can generate beat saber songs using the new automapper [InfernoSaber](https://github.com/fred-brenner/BeatSaber-Extraction)

Thanks to guillefix [DeepSaber](https://github.com/oxai/deepsaber) for the notebook layout.


**Run this every time you start the Colab session:**

Run the "Prepare dependencies" cell

**To generate each song:**

Follow the instructions below to upload song and generate levels

It should typically take 20 to 40 seconds depending on length of song (with GPU).

In [ ]:
#@title Connect Google Drive (double click to see code)
#@markdown This will connect colab to your Google Drive (will ask you for permission). Make sure you have more than a 1 GB free on your Google Drive (to be safe)

#@markdown It will then create a folder to save the InfernoSaber depencies
#@markdown or if it's already created it will use this name to find the folder


from google.colab import drive, files
drive.mount('/content/drive')
import os
from os import mkdir, path
def cd(dir):
  os.chdir(dir)
infernosaber_folder_name = 'InfernoSaber'
working_dir=!pwd
working_dir=working_dir[0]
if working_dir=="/content":
  if not path.exists("drive/My Drive/"+infernosaber_folder_name):
    mkdir("drive/My Drive/"+infernosaber_folder_name)
  cd("drive/My Drive/"+infernosaber_folder_name)

#@markdown **run this cell** by pressing the play arrow on the left (or press ctrl+enter or shift+enter while having this cell selected)

In [ ]:
#@title Prepare InfernoSaber and dependencies
#@markdown Run this cell to download deepsaber to your drive (only will if it hasn't downloaded already), and prepare dependencies

if len(os.listdir()) > 0:
  print("Code already setup")
else:
  print("Download pretrained package.")
  !wget -O 'InfernoSaber.zip' 'https://docs.google.com/uc?export=download&id=1crgCp5Ey4Ts8iVNf32JiEywSsi0rCzYB&confirm=t'
  import zipfile
  with zipfile.ZipFile('InfernoSaber.zip', 'r') as zip_ref:
    zip_ref.extractall('')
  os.remove('InfernoSaber.zip')
  print("Finished code setup")

# install needed python modules in notebook
print("Installing music dependencies...")
!pip install scikit-learn==1.0.2 aubio pydub keras-tcn --quiet
# tensorflow related modules are currently installed on every colab notebook
print("Finished environment setup. Ready for takeoff!")

## Generate

In [ ]:
#@title Local song upload
#@markdown Upload a song (MP3, MP4, OGG, EGG supported) or

#@markdown You can skip this step and directly copy the music into the Drive folder 
#@markdown "/content/drive/MyDrive/InfernoSaber/Data/prediction/songs_predict/"

#@markdown You choose multiple songs at once or rerun this cell to import multiple songs

#@markdown You can choose to delete 1.) old input songs, 2.) old output maps

# Optional: delete all files in directory
song_path = 'Data/prediction/songs_predict/'
del_flag = input("Delete previous input songs? (y or n)")
if del_flag.lower() == 'y':
  for f in os.listdir(song_path):
    if not f.startswith('Put your music files HERE.txt'):
      os.remove(song_path + f)
del_flag = input("Delete previous output maps? (y or n)")
if del_flag.lower() == 'y':
  map_path = 'Data/prediction/new_map/'
  for f in os.listdir(map_path):
    if f not in ["cover.jpg", "Output directory.txt"]:
      if os.path.isfile(map_path + f):
        os.remove(map_path + f)
      else:
        import shutil
        shutil.rmtree(map_path + f)

# save new music file(s)
res=files.upload()
filename=list(res.keys())
for song_file in filename:
  os.rename(song_file, song_path + song_file)

In [ ]:
#@title Youtube song download

#@markdown Library by [pytube](https://github.com/rmerzouki/pytube)

#@markdown You can rerun this cell to import multiple songs

!pip install pytube --quiet
from pytube import YouTube

song_path = 'Data/prediction/songs_predict/'

download_from_youtube = True #@param {type:"boolean"}
link = 'https://www.youtube.com/watch?v=fr-xxBY_BdQ&ab_channel=CamelliaOfficial' #@param {type:"string"}

if download_from_youtube:
  if (link.startswith("https://youtu.be/")): # fix
    link = link.replace("https://youtu.be/", "https://www.youtube.com/watch?v=")

  # Create a YouTube object
  video = YouTube(link)
  audio_streams = video.streams.filter(only_audio=True)
  # Select the first audio stream from the list (which is usually the one with the highest audio quality)
  audio_stream = audio_streams[0]
  audio_stream.download(output_path=song_path)


In [5]:
#@title Specify mapper parameters
#@markdown Set the maximum difficulty and further parameters by adjusting in the code (double click to see code).

# recommended: 15 for hard, 25 for Expert+; set between 5 for easy difficulty to 40 for expert++
os.environ['MAX_SPEED'] = '34'
os.environ['quick_start'] = '1.8'


In [ ]:
#@title Generate level for song
#@markdown Output will be put in the new map folder (both zip and extracted): "/content/drive/MyDrive/InfernoSaber/Data/prediction/new_map" 
#@markdown

#@markdown Old maps will stay except for 1) same song again 2) removed in step "Local song upload"

#@markdown Download file by right click and press download or go into the google drive to download all.

# start main script
!python "Code/main.py"

# files.download("scripts/generation/generated/"+generated_filename)

You can have a view on the mapped songs at "/content/drive/MyDrive/InfernoSaber/Data/prediction/new_map" 

You can upload the generated zip file to view the map online here: https://skystudioapps.com/bs-viewer/
(thanks to +1 Rabbit).